In [11]:
import pandas as pd
from pathlib import Path

In [13]:
file1 = Path("Resources/schools_complete.csv")
file2 = Path("Resources/students_complete.csv")

In [15]:
df_school = pd.read_csv(file1)
df_school.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [17]:
df_students = pd.read_csv(file2)
df_students.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [19]:
all_school_data = pd.merge(df_students, df_school, how= "left", on=["school_name"])
all_school_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [21]:
list_schools = all_school_data["school_name"].nunique()
list_schools

15

In [23]:
total_students = all_school_data["student_name"].count()
total_students

39170

In [25]:
u_budgets = all_school_data["budget"].unique()
total_budget = u_budgets.sum()
total_budget

24649428

In [27]:
average_reading = all_school_data["reading_score"].mean()
average_reading

81.87784018381414

In [29]:
average_math = all_school_data["math_score"].mean()
average_math

78.98537145774827

In [31]:
pass_math_count = all_school_data[(all_school_data["math_score"] >=70)].count()["student_name"]
porcentage_pass_math = pass_math_count / float(total_students) * 100
porcentage_pass_math

74.9808526933878

In [33]:
pass_reading_count = all_school_data[(all_school_data["reading_score"] >=70)].count()["student_name"]
porcentage_pass_reading = pass_reading_count / float(total_students) * 100
porcentage_pass_reading

85.80546336482001

In [35]:
pass_reading_math_count = all_school_data[
(all_school_data["reading_score"] >=70) & (all_school_data["math_score"] >=70)].count()["student_name"]
pass_reading_math_porcentage = pass_reading_math_count / float(total_students) * 100
pass_reading_math_porcentage

65.17232575950983

In [37]:
summary_data = {"Total Students": [total_students], "Total Schools": [list_schools], "Total Budget": [total_budget],
                "Average Reading Score": [average_reading], "Average Math Score": [average_math], "% Passing Math": [porcentage_pass_math],
                "% Passing Reading": [porcentage_pass_reading], "% Overall Passing": [pass_reading_math_porcentage]}
district_summary = pd.DataFrame(summary_data)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary

,Total Students,Total Schools,Total Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,"39,170",15,"$24,649,428.00",81.87784,78.985371,74.980853,85.805463,65.172326


In [39]:
school_types = all_school_data.set_index(["school_name"])["type"]

In [41]:
total_stper_school = all_school_data.groupby("school_name")["size"].first().reset_index(name="Size")

In [43]:
total_school_budget = all_school_data.groupby("school_name")["budget"].first().reset_index(name="Budget")

In [45]:
school_capita = all_school_data["budget"]/ all_school_data["size"]
per_school_capita_df = pd.DataFrame({"school_name": all_school_data["school_name"], "per_capita": school_capita})
per_school_capita= per_school_capita_df.groupby("school_name").mean().reset_index()
per_school_capita = per_school_capita.rename(columns={"per_capita": "Per capita"})

In [47]:
school_reading_average = all_school_data.groupby("school_name")["reading_score"].mean().reset_index(name="Reading Average")

In [49]:
school_math_average = all_school_data.groupby("school_name")["math_score"].mean().reset_index(name="Math Average")

In [51]:
school_pass_math_count = all_school_data[(all_school_data["math_score"] >=70)].groupby("school_name").size().reset_index(name="Math Count")

In [53]:
school_pass_reading_count = all_school_data[(all_school_data["reading_score"] >=70)].groupby("school_name").size().reset_index(name="Read Count")

In [55]:
school_passing_math_and_reading = all_school_data[
    (all_school_data["reading_score"] >= 70) & 
(all_school_data["math_score"] >= 70)].groupby("school_name").size().reset_index(name="Math Read Count")

In [197]:
school_summary_data = {"Total School Budget":total_school_budget["Budget"],"Total Students": total_stper_school["Size"],
                       "Per Student Budget": per_school_capita["Per capita"], "Average Reading score":school_reading_average["Reading Average"],
                       "Average Math score":school_math_average["Math Average"], "% Passing Math":school_pass_math_count["Math Count"] / total_stper_school["Size"] * 100, 
                      "% Passing Reading":school_pass_reading_count["Read Count"] / total_stper_school["Size"] * 100,
                       "% Passing Overall":school_passing_math_and_reading["Math Read Count"] / total_stper_school["Size"] * 100}
school_summary = pd.DataFrame(school_summary_data)
school_types = all_school_data.set_index(["school_name"])["type"]

In [199]:
school_summary["Total School Budget"] = school_summary["Total School Budget"].map("${:,.2f}".format)
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].map("${:,.2f}".format)
school_summary

,Total School Budget,Total Students,Per Student Budget,Average Reading score,Average Math score,% Passing Math,% Passing Reading,% Passing Overall
0,"$3,124,928.00",4976,$628.00,81.033963,77.048432,66.680064,81.933280,54.642283
1,"$1,081,356.00",1858,$582.00,83.975780,83.061895,94.133477,97.039828,91.334769
2,"$1,884,411.00",2949,$639.00,81.158020,76.711767,65.988471,80.739234,53.204476
3,"$1,763,916.00",2739,$644.00,80.746258,77.102592,68.309602,79.299014,54.289887
4,"$917,500.00",1468,$625.00,83.816757,83.351499,93.392371,97.138965,90.599455
5,"$3,022,020.00",4635,$652.00,80.934412,77.289752,66.752967,80.862999,53.527508
6,"$248,087.00",427,$581.00,83.814988,83.803279,92.505855,96.252927,89.227166
7,"$1,910,635.00",2917,$655.00,81.182722,76.629414,65.683922,81.316421,53.513884
8,"$3,094,650.00",4761,$650.00,80.966394,77.072464,66.057551,81.222432,53.539172
9,"$585,858.00",962,$609.00,84.044699,83.839917,94.594595,95.945946,90.540541


In [61]:
highest_performing_schools = school_summary.sort_values(by="% Passing Overall", ascending=False)
highest_performing_schools.head(5)

,Total School Budget,Total Students,Per Student Budget,Average Reading score,Average Math score,% Passing Math,% Passing Reading,% Passing Overall
1,"$1,081,356.00",1858,$582.00,83.975780,83.061895,94.133477,97.039828,91.334769
12,"$1,043,130.00",1635,$638.00,83.848930,83.418349,93.272171,97.308869,90.948012
4,"$917,500.00",1468,$625.00,83.816757,83.351499,93.392371,97.138965,90.599455
13,"$1,319,574.00",2283,$578.00,83.989488,83.274201,93.867718,96.539641,90.582567
9,"$585,858.00",962,$609.00,84.044699,83.839917,94.594595,95.945946,90.540541


In [63]:
highest_performing_schools = school_summary.sort_values(by="% Passing Overall", ascending=True)
highest_performing_schools.head(5)

,Total School Budget,Total Students,Per Student Budget,Average Reading score,Average Math score,% Passing Math,% Passing Reading,% Passing Overall
10,"$2,547,363.00",3999,$637.00,80.744686,76.842711,66.366592,80.220055,52.988247
2,"$1,884,411.00",2949,$639.00,81.158020,76.711767,65.988471,80.739234,53.204476
7,"$1,910,635.00",2917,$655.00,81.182722,76.629414,65.683922,81.316421,53.513884
5,"$3,022,020.00",4635,$652.00,80.934412,77.289752,66.752967,80.862999,53.527508
8,"$3,094,650.00",4761,$650.00,80.966394,77.072464,66.057551,81.222432,53.539172


In [65]:
per_school_passing_math = school_pass_math_count["Math Count"] / total_stper_school["Size"] * 100
per_school_passing_reading = school_pass_reading_count["Read Count"] / total_stper_school["Size"] * 100
overall_passing_rate = school_passing_math_and_reading["Math Read Count"] / total_stper_school["Size"] * 100

In [67]:
ninth_graders = all_school_data[(all_school_data["grade"] == "9th")]
tenth_graders = all_school_data[(all_school_data["grade"] == "10th")]
eleventh_graders = all_school_data[(all_school_data["grade"] == "11th")]
twelfth_graders = all_school_data[(all_school_data["grade"] == "12th")]

In [69]:
mean_math_scores_by_grade = all_school_data.groupby(["school_name", "grade"])["math_score"].mean().reset_index(name="Mean Math Score")
math_scores_by_grade = pd.DataFrame(mean_math_scores_by_grade)

In [71]:
mean_math_scores_ninth = ninth_graders.groupby("school_name")["math_score"].mean().reset_index(name="Ninth Math Score")
mean_math_scores_tenth = tenth_graders.groupby("school_name")["math_score"].mean().reset_index(name="Tenth Math Score")
mean_math_scores_eleventh = eleventh_graders.groupby("school_name")["math_score"].mean().reset_index(name="Eleventh Math Score")
mean_math_scores_twelfth = twelfth_graders.groupby("school_name")["math_score"].mean().reset_index(name="Twelfth Math Score")

In [73]:
math_scores_by_grade_data = {"9th": mean_math_scores_ninth["Ninth Math Score"], "10th": mean_math_scores_tenth["Tenth Math Score"], 
                             "11th": mean_math_scores_eleventh["Eleventh Math Score"], "12th": mean_math_scores_twelfth["Twelfth Math Score"]}
math_scores_by_grade = pd.DataFrame(math_scores_by_grade_data)
math_scores_by_grade.index.name = None
math_scores_by_grade

,9th,10th,11th,12th
0,77.083676,76.996772,77.515588,76.492218
1,83.094697,83.154506,82.765560,83.277487
2,76.403037,76.539974,76.884344,77.151369
3,77.361345,77.672316,76.918058,76.179963
4,82.044010,84.229064,83.842105,83.356164
5,77.438495,77.337408,77.136029,77.186567
6,83.787402,83.429825,85.000000,82.855422
7,77.027251,75.908735,76.446602,77.225641
8,77.187857,76.691117,77.491653,76.863248
9,83.625455,83.372000,84.328125,84.121547


In [75]:
mean_reading_scores_ninth = ninth_graders.groupby("school_name")["reading_score"].mean().reset_index(name="Ninth Reading Score")
mean_reading_scores_tenth = tenth_graders.groupby("school_name")["reading_score"].mean().reset_index(name="Tenth Reading Score")
mean_reading_scores_eleventh = eleventh_graders.groupby("school_name")["reading_score"].mean().reset_index(name="Eleventh Reading Score")
mean_reading_scores_twelfth = twelfth_graders.groupby("school_name")["reading_score"].mean().reset_index(name="Twelfth Reading Score")

In [77]:
math_scores_by_grade_data = {"9th": mean_reading_scores_ninth["Ninth Reading Score"], "10th": mean_reading_scores_tenth["Tenth Reading Score"], 
                             "11th": mean_reading_scores_eleventh["Eleventh Reading Score"], "12th": mean_reading_scores_twelfth["Twelfth Reading Score"]}
math_scores_by_grade = pd.DataFrame(math_scores_by_grade_data)
math_scores_by_grade.index.name = None
math_scores_by_grade

,9th,10th,11th,12th
0,81.303155,80.907183,80.945643,80.912451
1,83.676136,84.253219,83.788382,84.287958
2,81.198598,81.408912,80.640339,81.384863
3,80.632653,81.262712,80.403642,80.662338
4,83.369193,83.706897,84.288089,84.013699
5,80.866860,80.660147,81.396140,80.857143
6,83.677165,83.324561,83.815534,84.698795
7,81.290284,81.512386,81.417476,80.305983
8,81.260714,80.773431,80.616027,81.227564
9,83.807273,83.612000,84.335938,84.591160


In [105]:
spending_bins = [0, 585, 630, 645, 680]
ranges = ["<$585", "$585-630", "$630-645", "$645-680"]
school_spending_df = school_summary.copy()
school_summary["Per Student Budget"] = school_summary["Per Student Budget"].replace('[\$,]', '', regex=True).astype(float)

In [116]:
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=ranges,
                                                             include_lowest=True)
school_spending_df

,Total School Budget,Total Students,Per Student Budget,Average Reading score,Average Math score,% Passing Math,% Passing Reading,% Passing Overall,Spending Ranges (Per Student)
0,"$3,124,928.00",4976,$628.00,81.033963,77.048432,66.680064,81.933280,54.642283,$585-630
1,"$1,081,356.00",1858,$582.00,83.975780,83.061895,94.133477,97.039828,91.334769,<$585
2,"$1,884,411.00",2949,$639.00,81.158020,76.711767,65.988471,80.739234,53.204476,$630-645
3,"$1,763,916.00",2739,$644.00,80.746258,77.102592,68.309602,79.299014,54.289887,$630-645
4,"$917,500.00",1468,$625.00,83.816757,83.351499,93.392371,97.138965,90.599455,$585-630
5,"$3,022,020.00",4635,$652.00,80.934412,77.289752,66.752967,80.862999,53.527508,$645-680
6,"$248,087.00",427,$581.00,83.814988,83.803279,92.505855,96.252927,89.227166,<$585
7,"$1,910,635.00",2917,$655.00,81.182722,76.629414,65.683922,81.316421,53.513884,$645-680
8,"$3,094,650.00",4761,$650.00,80.966394,77.072464,66.057551,81.222432,53.539172,$645-680
9,"$585,858.00",962,$609.00,84.044699,83.839917,94.594595,95.945946,90.540541,$585-630


In [120]:
size_bins = [0, 1000, 2000, 5000]
size_ranges = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [128]:
school_summary["School Size"] =pd.cut(school_summary["Total Students"], size_bins, labels=size_ranges,
                                                             include_lowest=True)
school_summary

,Total School Budget,Total Students,Per Student Budget,Average Reading score,Average Math score,% Passing Math,% Passing Reading,% Passing Overall,School Size
0,"$3,124,928.00",4976,628.0,81.033963,77.048432,66.680064,81.933280,54.642283,Large (2000-5000)
1,"$1,081,356.00",1858,582.0,83.975780,83.061895,94.133477,97.039828,91.334769,Medium (1000-2000)
2,"$1,884,411.00",2949,639.0,81.158020,76.711767,65.988471,80.739234,53.204476,Large (2000-5000)
3,"$1,763,916.00",2739,644.0,80.746258,77.102592,68.309602,79.299014,54.289887,Large (2000-5000)
4,"$917,500.00",1468,625.0,83.816757,83.351499,93.392371,97.138965,90.599455,Medium (1000-2000)
5,"$3,022,020.00",4635,652.0,80.934412,77.289752,66.752967,80.862999,53.527508,Large (2000-5000)
6,"$248,087.00",427,581.0,83.814988,83.803279,92.505855,96.252927,89.227166,Small (<1000)
7,"$1,910,635.00",2917,655.0,81.182722,76.629414,65.683922,81.316421,53.513884,Large (2000-5000)
8,"$3,094,650.00",4761,650.0,80.966394,77.072464,66.057551,81.222432,53.539172,Large (2000-5000)
9,"$585,858.00",962,609.0,84.044699,83.839917,94.594595,95.945946,90.540541,Small (<1000)


In [167]:
size_math_scores = school_summary.groupby("School Size")["Average Math score"].mean().reset_index(name="Size Math Score")
size_reading_scores = school_summary.groupby("School Size")["Average Reading score"].mean().reset_index(name="Size Reading Score")
size_passing_reading = school_summary.groupby("School Size")["% Passing Math"].mean().reset_index(name="Size Math Pass")
size_passing_math = school_summary.groupby("School Size")["% Passing Reading"].mean().reset_index(name="Size Reading Pass")
size_passing_overall = school_summary.groupby("School Size")["% Passing Overall"].mean().reset_index(name="Size Passing Overall")

C:\Users\Santiago\AppData\Local\Temp\ipykernel_1736\4289671755.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_math_scores = school_summary.groupby("School Size")["Average Math score"].mean().reset_index(name="Size Math Score")
C:\Users\Santiago\AppData\Local\Temp\ipykernel_1736\4289671755.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  size_reading_scores = school_summary.groupby("School Size")["Average Reading score"].mean().reset_index(name="Size Reading Score")
C:\Users\Santiago\AppData\Local\Temp\ipykernel_1736\4289671755.py:3: FutureWarning: The default of observed=False is deprecated and w

In [191]:
size_summary_data = {"Average Math score":size_math_scores["Size Math Score"], "Average Reading score":size_reading_scores["Size Reading Score"], 
                     "% Passing Math":size_passing_reading["Size Math Pass"], "% Passing Reading":size_passing_math["Size Reading Pass"], 
                     "% Passing Overall":size_passing_overall["Size Passing Overall"]}
size_summary = pd.DataFrame(size_summary_data)
size_summary

,Average Math score,Average Reading score,% Passing Math,% Passing Reading,% Passing Overall
0,83.821598,83.929843,93.550225,96.099437,89.883853
1,83.374684,83.864438,93.599695,96.790680,90.621535
2,77.746417,81.344493,69.963361,82.766634,58.286003
